In [1]:
# %load_ext tensorboard
# Clear any logs from previous runs
# !rm -rf ./logs/

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd
np.random.seed(0)

2022-12-16 10:58:25.285037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 10:58:25.455756: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 10:58:25.455792: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-16 10:58:26.520187: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
url = 'https://raw.githubusercontent.com/igomezv/nnogada/main/data/jla.csv'
df=pd.read_csv(url)
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)

(740, 2)

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.8
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((592, 1), (148, 1), (592, 2), (148, 2))

In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=200,
                                   restore_best_weights=True, verbose=True)
                                   ]

n_cols = 1

In [7]:
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([2, 4, 8, 16]))
HP_LAYERS =    hp.HParam('layers', hp.Discrete([1, 2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-4,1e-3]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))

In [8]:
epochs=200

In [9]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [10]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning3').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING, 
                 HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

2022-12-16 10:58:28.071654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 10:58:28.071876: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 10:58:28.071931: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-16 10:58:28.071980: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-16 10:58:28.072024: W tensorflow/c

In [11]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Dense(int(X_train.shape[1])))
#     model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams[HP_BATCHSIZE], shuffle=False, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [12]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [13]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
            for batch_size in HP_BATCHSIZE.domain.values:
                t = time.time()
                hparams = {

                    HP_LAYERS: deep_layers,
                    HP_NUM_UNITS: num_units,
                    HP_LEARNING: learning_rate,
                    HP_BATCHSIZE: batch_size,
                }
                run_name = "run-%d" % session_num
                print('\n--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                score = run('logs/hparam_tuning3/' + run_name, hparams)
                t = time.time()-t
                session_num += 1
                print("Loss:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, batch_size, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0626 - mean_squared_error: 0.0626
Loss: 0.0626378282904625 Tiempo transcurrido: 100.73911809921265

--- Starting trial: run-1
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.1157 - mean_squared_error: 0.1157
Loss: 0.11571776121854782 Tiempo transcurrido: 52.81896448135376

--- Starting trial: run-2
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.1855 - mean_squared_error: 0.1855
Loss: 0.18548725545406342 Tiempo transcurrido: 28.00128936767578

--- Starting trial: run-3
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 0.9441 - mean_squared_error: 0.9441
Loss: 0.944054901599884 

5/5 [==============================] - 0s 2ms/step - loss: 0.0361 - mean_squared_error: 0.0361
Loss: 0.03606678918004036 Tiempo transcurrido: 21.493022203445435

--- Starting trial: run-32
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0331 - mean_squared_error: 0.0331
Loss: 0.03311280161142349 Tiempo transcurrido: 111.81642723083496

--- Starting trial: run-33
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 3ms/step - loss: 0.0372 - mean_squared_error: 0.0372
Loss: 0.03720268979668617 Tiempo transcurrido: 64.16965508460999

--- Starting trial: run-34
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 3ms/step - loss: 0.0526 - mean_squared_error: 0.0526
Loss: 0.0526273176074028 Tiempo transcurrido: 40.551892042160034

--- Starting trial: run-35
{'layers': 2, 'num_units': 

5/5 [==============================] - 0s 1ms/step - loss: 0.0324 - mean_squared_error: 0.0324
Loss: 0.03239349275827408 Tiempo transcurrido: 37.56405830383301

--- Starting trial: run-94
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 1ms/step - loss: 0.0335 - mean_squared_error: 0.0335
Loss: 0.033487483859062195 Tiempo transcurrido: 22.547380447387695

--- Starting trial: run-95
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0412 - mean_squared_error: 0.0412
Loss: 0.04124600067734718 Tiempo transcurrido: 14.403990507125854

--- Starting trial: run-96
{'layers': 4, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 1ms/step - loss: 0.0304 - mean_squared_error: 0.0304
Loss: 0.030388806015253067 Tiempo transcurrido: 61.15305709838867

--- Starting trial: run-97
{'layers': 4, 'num_units

In [14]:
filename = "historial_jla_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Batch size", "MSE", "Tiempo de ejecución"])

df.sort_values(by=["MSE", "Tiempo de ejecución"], ascending=[True, True], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [15]:
df.head(2)

,Deep size,Num units,Learning rate,Batch size,MSE,Tiempo de ejecución
0,4,150,0.0001,16,0.032538,15.633505
1,4,100,0.0010,16,0.033401,14.066662


In [16]:
total_time = float(np.sum(df[["Tiempo de ejecución"]])/60)

print("Elapsed time {:.3f} minutes".format(total_time))

Elapsed time 8.939 minutes


In [17]:
df["Tiempo de ejecución"]

0     15.633505
1     14.066662
2     23.042750
3     18.078668
4     21.493022
5     13.890556
6     14.407193
7     15.622892
8     13.899126
9     13.206993
10    18.320965
11    13.261669
12    14.403991
13    19.112864
14    17.940388
15    20.161535
16    13.060703
17    13.448699
18    14.322025
19    12.674035
20    16.227475
21    16.694944
22    13.601906
23    15.724212
24    12.715001
25    13.430503
26    22.064104
27    24.203935
28    21.514397
29    24.894202
30    17.672660
31    17.556799
Name: Tiempo de ejecución, dtype: float64